In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 1. Dataset Preparation
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to float32 tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

# Load CIFAR-100 dataset
train_dataset = datasets.CIFAR100(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR100(root="./data", train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check data types (Optional for debugging)
data_iter = iter(train_loader)
images, labels = next(data_iter)
print(f"Image dtype: {images.dtype}, Label dtype: {labels.dtype}")

# 2. Define Model
class ModifiedResNet(nn.Module):
    def __init__(self, num_classes=100):
        super(ModifiedResNet, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.model.fc = nn.Sequential(
            nn.Linear(self.model.fc.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Instantiate model
model = ModifiedResNet(num_classes=100).to(device)

# 3. Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Training Function
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device, dtype=torch.float32), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

# Train the model
train_model(model, train_loader, criterion, optimizer, epochs=5)

# 5. Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device, dtype=torch.float32), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    return y_true, y_pred

# Evaluate the model
y_true, y_pred = evaluate_model(model, test_loader)

# 6. Calculate Metrics
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")
print(classification_report(y_true, y_pred))

# 7. Save and Load Model
torch.save(model.state_dict(), "cifar100_resnet50.pth")
print("Model saved!")

# To load the model later:
# model.load_state_dict(torch.load("cifar100_resnet50.pth"))
# model.to(device)


Using device: cpu
Files already downloaded and verified
Files already downloaded and verified
Image dtype: torch.float32, Label dtype: torch.int64
